In [44]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder


In [45]:
# Load the data. 
# Used zip file bcse csv file was too big to upload to github.
df = pd.read_csv('../Resources/2017_raw.csv')
df.head()

,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,...,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 27
0,2017-01-01,AA,1,JFK,LAX,800,831.0,31.0,25.0,856.0,...,402.0,398.0,347.0,2475.0,27.0,0.0,0.0,0.0,0.0,NaN
1,2017-01-01,AA,2,LAX,JFK,900,934.0,34.0,34.0,1008.0,...,327.0,335.0,289.0,2475.0,34.0,0.0,8.0,0.0,0.0,NaN
2,2017-01-01,AA,4,LAX,JFK,1130,1221.0,51.0,20.0,1241.0,...,328.0,319.0,284.0,2475.0,7.0,0.0,0.0,0.0,35.0,NaN
3,2017-01-01,AA,5,DFW,HNL,1135,1252.0,77.0,19.0,1311.0,...,517.0,537.0,513.0,3784.0,77.0,0.0,20.0,0.0,0.0,NaN
4,2017-01-01,AA,6,OGG,DFW,1855,1855.0,0.0,16.0,1911.0,...,425.0,467.0,440.0,3711.0,0.0,0.0,42.0,0.0,0.0,NaN


In [46]:
df.OP_CARRIER.unique()

array(['AA', 'B6', 'EV', 'HA', 'NK', 'OO', 'UA', 'VX', 'AS', 'WN', 'DL',
       'F9'], dtype=object)

In [47]:
df['OP_CARRIER'].replace({
    'UA':'United Airlines',
    'AS':'Alaska Airlines',
    '9E':'Endeavor Air',
    'B6':'JetBlue Airways',
    'EV':'ExpressJet',
    'F9':'Frontier Airlines',
    'G4':'Allegiant Air',
    'HA':'Hawaiian Airlines',
    'MQ':'Envoy Air',
    'NK':'Spirit Airlines',
    'OH':'PSA Airlines',
    'OO':'SkyWest Airlines',
    'VX':'Virgin America',
    'WN':'Southwest Airlines',
    'YV':'Mesa Airline',
    'YX':'Republic Airways',
    'AA':'American Airlines',
    'DL':'Delta Airlines'
}, inplace=True)

In [48]:
df.OP_CARRIER.unique()

array(['American Airlines', 'JetBlue Airways', 'ExpressJet',
       'Hawaiian Airlines', 'Spirit Airlines', 'SkyWest Airlines',
       'United Airlines', 'Virgin America', 'Alaska Airlines',
       'Southwest Airlines', 'Delta Airlines', 'Frontier Airlines'],
      dtype=object)

In [49]:
#Deleting cancelled flights
df_new = df[df['CANCELLED']==0]


In [50]:
df['CANCELLED'].unique()

array([0., 1.])

In [51]:
print(f'Total rows = {len(df_new)}')

Total rows = 5591928


In [52]:
df.OP_CARRIER.value_counts()

Southwest Airlines    1329444
Delta Airlines         923560
American Airlines      896348
SkyWest Airlines       706527
United Airlines        584481
ExpressJet             339541
JetBlue Airways        298654
Alaska Airlines        185068
Spirit Airlines        156818
Frontier Airlines      103027
Hawaiian Airlines       80172
Virgin America          70981
Name: OP_CARRIER, dtype: int64

In [53]:
df.head()

,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,...,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 27
0,2017-01-01,American Airlines,1,JFK,LAX,800,831.0,31.0,25.0,856.0,...,402.0,398.0,347.0,2475.0,27.0,0.0,0.0,0.0,0.0,NaN
1,2017-01-01,American Airlines,2,LAX,JFK,900,934.0,34.0,34.0,1008.0,...,327.0,335.0,289.0,2475.0,34.0,0.0,8.0,0.0,0.0,NaN
2,2017-01-01,American Airlines,4,LAX,JFK,1130,1221.0,51.0,20.0,1241.0,...,328.0,319.0,284.0,2475.0,7.0,0.0,0.0,0.0,35.0,NaN
3,2017-01-01,American Airlines,5,DFW,HNL,1135,1252.0,77.0,19.0,1311.0,...,517.0,537.0,513.0,3784.0,77.0,0.0,20.0,0.0,0.0,NaN
4,2017-01-01,American Airlines,6,OGG,DFW,1855,1855.0,0.0,16.0,1911.0,...,425.0,467.0,440.0,3711.0,0.0,0.0,42.0,0.0,0.0,NaN


In [55]:
#Dropping the columns which are not required
df = df.drop(["FL_DATE","OP_CARRIER_FL_NUM","DEP_TIME","ARR_TIME","CANCELLED","CANCELLATION_CODE","DIVERTED","CARRIER_DELAY"
             ,"WEATHER_DELAY","NAS_DELAY","SECURITY_DELAY","LATE_AIRCRAFT_DELAY"], axis=1)

In [58]:
df = df.drop(["Unnamed: 27"], axis=1)

In [59]:
for col in df.columns:
    print(col)

OP_CARRIER
ORIGIN
DEST
CRS_DEP_TIME
DEP_DELAY
TAXI_OUT
WHEELS_OFF
WHEELS_ON
TAXI_IN
CRS_ARR_TIME
ARR_DELAY
CRS_ELAPSED_TIME
ACTUAL_ELAPSED_TIME
AIR_TIME
DISTANCE


In [60]:
df.head()

,OP_CARRIER,ORIGIN,DEST,CRS_DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_DELAY,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE
0,American Airlines,JFK,LAX,800,31.0,25.0,856.0,1143.0,26.0,1142,27.0,402.0,398.0,347.0,2475.0
1,American Airlines,LAX,JFK,900,34.0,34.0,1008.0,1757.0,12.0,1727,42.0,327.0,335.0,289.0,2475.0
2,American Airlines,LAX,JFK,1130,51.0,20.0,1241.0,2025.0,15.0,1958,42.0,328.0,319.0,284.0,2475.0
3,American Airlines,DFW,HNL,1135,77.0,19.0,1311.0,1744.0,5.0,1612,97.0,517.0,537.0,513.0,3784.0
4,American Airlines,OGG,DFW,1855,0.0,16.0,1911.0,631.0,11.0,600,42.0,425.0,467.0,440.0,3711.0


In [61]:
print(f'Total rows = {len(df_new)}')

Total rows = 5591928


In [63]:
df = df.dropna()

In [64]:
print(f'Total rows = {len(df_new)}')

Total rows = 5591928


In [66]:
df.isna().sum()

OP_CARRIER             0
ORIGIN                 0
DEST                   0
CRS_DEP_TIME           0
DEP_DELAY              0
TAXI_OUT               0
WHEELS_OFF             0
WHEELS_ON              0
TAXI_IN                0
CRS_ARR_TIME           0
ARR_DELAY              0
CRS_ELAPSED_TIME       0
ACTUAL_ELAPSED_TIME    0
AIR_TIME               0
DISTANCE               0
dtype: int64